In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

import torchvision.datasets as datasets
import torchvision.transforms as transforms

import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"

In [34]:
class NN(nn.Module):
    def __init__(self,input_size,num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 2)
        self.fc2 = nn.Linear(2, num_classes)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [35]:
input_size = 2
num_classes = 3
learning_rate = 0.001
batch_size = 64
num_epochs = 1

In [36]:
# train_dataset = datasets.MNIST('./dataset', train=True, transform=transforms.ToTensor(),download=True)
# train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)  

# test_dataset = datasets.MNIST('./dataset', train=False, transform=transforms.ToTensor(),download=True)
# test_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)  

df = pd.read_csv('ModelGrader/modelgrader_submission_processed.csv',index_col='problem_id')

X_train, X_test, y_train, y_test = train_test_split(df[["avg_first_passed_total_attempts","avg_first_passed_time_used"]], df["difficulty"], test_size=0.2, random_state=42)

X_train = torch.tensor(X_train.values, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train.values, dtype=torch.long).to(device)

X_test = torch.tensor(X_test.values, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test.values, dtype=torch.long).to(device)

In [37]:
model = NN(input_size=input_size,num_classes=num_classes).to(device)

In [38]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [39]:
def check_accuracy(loader,model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x,y in loader:
            x = x.to(device)
            y = y.to(device)
            x = x.reshape(x.shape[0],-1)

            scores = model(x)
            _,predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        acc = float(num_correct)/float(num_samples)*100
        print(f"{acc:.2f}")
    
    model.train()
    return acc

In [40]:
for epoch in range(num_epochs):
    for item in X_train:
        # data = data.to(device=device)
        # targets = targets.to(device=device)

        # data = data.reshape(data.shape[0],-1)

        score = model(item)
        loss = criterion(score,targets)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

NameError: name 'data' is not defined

: 

In [ ]:
check_accuracy(train_loader,model)
check_accuracy(test_loader,model)

93.16
93.16


93.16499999999999